# Replace DataSource Reference
Copyright (c) Microsoft Corporation. All rights reserved.<br>
Licensed under the MIT License.

A common practice when performing DataPrep is to build up a script or set of cleaning operations on a smaller example file locally. This is quicker and easier than dealing with large amounts of data initially.

After building a Dataflow that performs the desired steps, it's time to run it against the larger dataset, which may be stored in the cloud, or even locally just in a different file. This is where we can use `Dataflow.replace_datasource` to get a Dataflow identical to the one built on the small data, but referencing the newly specified DataSource.

In [1]:
import azureml.dataprep as dprep

dflow = dprep.read_csv('../data/crime-spring.csv')
df = dflow.to_pandas_dataframe()
df

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10498554,HZ239907,4/15/2016 23:56,007XX E 111TH ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,OTHER,FALSE,FALSE,...,9,50,11,1183356,1831503,2016,5/11/2016 15:48,41.69283384,-87.60431945,"(41.692833841, -87.60431945)"
1,10516598,HZ258664,4/15/2016 17:00,082XX S MARSHFIELD AVE,890,THEFT,FROM BUILDING,RESIDENCE,FALSE,FALSE,...,21,71,6,1166776,1850053,2016,5/12/2016 15:48,41.74410697,-87.66449429,"(41.744106973, -87.664494285)"
2,10519196,HZ261252,4/15/2016 10:00,104XX S SACRAMENTO AVE,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,RESIDENCE,FALSE,FALSE,...,19,74,11,,,2016,5/12/2016 15:50,,,
3,10519591,HZ261534,4/15/2016 9:00,113XX S PRAIRIE AVE,1120,DECEPTIVE PRACTICE,FORGERY,RESIDENCE,FALSE,FALSE,...,9,49,10,,,2016,5/13/2016 15:51,,,
4,10534446,HZ277630,4/15/2016 10:00,055XX N KEDZIE AVE,890,THEFT,FROM BUILDING,"SCHOOL, PUBLIC, BUILDING",FALSE,FALSE,...,40,13,6,,,2016,5/25/2016 15:59,,,
5,10535059,HZ278872,4/15/2016 4:30,004XX S KILBOURN AVE,810,THEFT,OVER $500,RESIDENCE,FALSE,FALSE,...,24,26,6,,,2016,5/25/2016 15:59,,,
6,10499802,HZ240778,4/15/2016 10:00,010XX N MILWAUKEE AVE,1152,DECEPTIVE PRACTICE,ILLEGAL USE CASH CARD,RESIDENCE,FALSE,FALSE,...,27,24,11,,,2016,5/27/2016 15:45,,,
7,10522293,HZ264802,4/15/2016 16:00,019XX W DIVISION ST,1110,DECEPTIVE PRACTICE,BOGUS CHECK,RESTAURANT,FALSE,FALSE,...,1,24,11,1163094,1908003,2016,5/16/2016 15:48,41.90320604,-87.67636193,"(41.903206037, -87.676361925)"
8,10523111,HZ265911,4/15/2016 8:00,061XX N SHERIDAN RD,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,FALSE,FALSE,...,48,77,11,,,2016,5/16/2016 15:50,,,
9,10525877,HZ268138,4/15/2016 15:00,023XX W EASTWOOD AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,,FALSE,FALSE,...,47,4,11,,,2016,5/18/2016 15:50,,,


Here we have the first 10 rows of a dataset called 'Crime'. The original dataset is over 100MB (admittedly not that large of a dataset but this is just an example).

We'll perform a few cleaning operations.

In [2]:
dflow_dropped = dflow.drop_columns(['Location', 'Updated On', 'X Coordinate', 'Y Coordinate', 'Description'])
sctb = dflow_dropped.builders.set_column_types()
sctb.learn(inference_arguments=dprep.InferenceArguments(day_first=False))
dflow_typed = sctb.to_dataflow()
dflow_typed.head(5)

,ID,Case Number,Date,Block,IUCR,Primary Type,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,Year,Latitude,Longitude
0,10498554.0,HZ239907,2016-04-15 23:56:00,007XX E 111TH ST,1153.0,DECEPTIVE PRACTICE,OTHER,False,False,531.0,5.0,9.0,50.0,11.0,2016.0,41.692834,-87.604319
1,10516598.0,HZ258664,2016-04-15 17:00:00,082XX S MARSHFIELD AVE,890.0,THEFT,RESIDENCE,False,False,614.0,6.0,21.0,71.0,6.0,2016.0,41.744107,-87.664494
2,10519196.0,HZ261252,2016-04-15 10:00:00,104XX S SACRAMENTO AVE,1154.0,DECEPTIVE PRACTICE,RESIDENCE,False,False,2211.0,22.0,19.0,74.0,11.0,2016.0,NaN,NaN
3,10519591.0,HZ261534,2016-04-15 09:00:00,113XX S PRAIRIE AVE,1120.0,DECEPTIVE PRACTICE,RESIDENCE,False,False,531.0,5.0,9.0,49.0,10.0,2016.0,NaN,NaN
4,10534446.0,HZ277630,2016-04-15 10:00:00,055XX N KEDZIE AVE,890.0,THEFT,"SCHOOL, PUBLIC, BUILDING",False,False,1712.0,17.0,40.0,13.0,6.0,2016.0,NaN,NaN


Now that we have a Dataflow with all our desired steps, we're ready to run against the 'full' dataset stored in Azure Blob.
All we need to do is pass the BlobDataSource into `replace_datasource` and we'll get back an identical Dataflow with the new DataSource substituted in.

In [3]:
dflow_replaced = dflow_typed.replace_datasource(dprep.BlobDataSource('https://dpreptestfiles.blob.core.windows.net/testfiles/crime0.csv'))

'replaced_dflow' will now pull data from the 168MB (729734 rows) version of Crime0.csv stored in Azure Blob!

NOTE: Dataflows can also be created by referencing a different Dataflow. Instead of using `replace_datasource`, there is a corresponding `replace_reference` method.

We should be careful now since pulling all that data down and putting it in a pandas dataframe isn't an ideal way to inspect the result of our Dataflow. So instead, to see that our steps are being applied to all the new data, we can add a `take_sample` step, which will select records at random (based on a given probability) to be returned.

The probability below takes the ~730000 rows down to a more inspectable ~73, though the number will vary each time `to_pandas_dataframe()` is run, since they are being randomly selected based on the probability.

In [4]:
dflow_random_sample= dflow_replaced.take_sample(probability=0.0001)
sample = dflow_random_sample.to_pandas_dataframe()
sample

,ID,Case Number,Date,Block,IUCR,Primary Type,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,Year,Latitude,Longitude
0,9882198.0,HX533372,None,024XX W CERMAK RD,560.0,ASSAULT,CTA BUS,False,False,1034.0,10.0,25.0,31.0,NaN,2014.0,41.851921,-87.685861
1,9625971.0,HX276498,None,015XX W 111TH ST,460.0,BATTERY,PARKING LOT/GARAGE(NON.RESID.),False,False,2234.0,22.0,34.0,75.0,NaN,2014.0,41.692230,-87.661557
2,9358491.0,HW501910,None,001XX N PARKSIDE AVE,486.0,BATTERY,SIDEWALK,False,True,1512.0,15.0,29.0,25.0,NaN,2013.0,41.882769,-87.766450
3,9315087.0,HW459118,None,063XX S DAMEN AVE,820.0,THEFT,OTHER,False,False,726.0,7.0,15.0,67.0,6.0,2013.0,41.779133,-87.674052
4,9253603.0,HW398556,None,074XX S MAY ST,1310.0,CRIMINAL DAMAGE,RESIDENCE-GARAGE,False,False,733.0,7.0,17.0,68.0,14.0,2013.0,41.759419,-87.652607
5,9191663.0,HW336507,None,008XX N LONG AVE,560.0,ASSAULT,APARTMENT,False,True,1524.0,15.0,37.0,25.0,NaN,2013.0,41.895892,-87.760820
6,9172834.0,HW317015,None,050XX W JACKSON BLVD,1320.0,CRIMINAL DAMAGE,STREET,False,False,1533.0,15.0,28.0,25.0,14.0,2013.0,41.876798,-87.752667
7,9080818.0,HW207014,None,006XX W OHARE ST,2093.0,NARCOTICS,GOVERNMENT BUILDING/PROPERTY,True,False,1654.0,16.0,41.0,76.0,26.0,2013.0,41.953791,-87.907473
8,8784007.0,HV457778,None,061XX N KEDZIE AVE,460.0,BATTERY,APARTMENT,True,False,2413.0,24.0,50.0,2.0,NaN,2012.0,41.993487,-87.709079
9,8763217.0,HV438355,None,027XX W 84TH ST,820.0,THEFT,VEHICLE NON-COMMERCIAL,True,False,835.0,8.0,18.0,70.0,6.0,2012.0,41.740815,-87.690676
